In [ ]:
# Import libraries
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from termcolor import colored
import os
import seaborn as sns

from sklearn.metrics import classification_report,confusion_matrix, ConfusionMatrixDisplay

import tensorflow as tf
import tensorflow.keras.layers as tfl

import tensorflow_datasets as tfds

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import concatenate, Conv2DTranspose
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation

from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.layers import Resizing, Rescaling
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from tensorflow.keras.applications import resnet50, vgg16, mobilenet_v2, MobileNetV2
from keras.applications.imagenet_utils import preprocess_input

In [ ]:
# split data
# https://towardsdatascience.com/how-to-split-a-tensorflow-dataset-into-train-validation-and-test-sets-526c8dd29438
def get_dataset_partitions_tf(dataset, train_split=0.6, val_split=0.2, test_split=0.2, seed=12):
    assert (train_split + test_split + val_split) == 1
    
    size = int(dataset.cardinality().numpy())
    
    # Specify seed to always have the same split distribution between runs
    dataset = dataset.shuffle(size, seed=seed)
    
    train_size = int(train_split * size)
    val_size = int(val_split * size)
    
    train_ds = dataset.take(train_size)   
    val_ds = dataset.skip(train_size).take(val_size)
    test_ds = dataset.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

def to_categorical(y, num_classes):
    y = tf.one_hot(tf.cast(y, tf.int32), num_classes)
    y = tf.cast(y, tf.float32)
    return y

def is_categorical(dataset):
    for _, y in dataset.take(1):
        output = y.numpy()[0]
        break
    if(len(output.shape) == 0):
        return False
    else:
        return True
    
def show_image(element, class_names):
    x = elem[0]
    y = elem[1]
    image = x[0].astype("uint8")
    idx = y[0]
    if(is_categorical(dataset)):
        idx = np.argmax(idx)
    plt.imshow(image)
    plt.title(class_names[idx])
    plt.show()    

def visualize_dataset(dataset, class_names, title):
    # visualize
    categorical = is_categorical(dataset)
    plt.figure(figsize=(10, 10))
    elems = list(dataset.as_numpy_iterator())
    for i in range(9):
        image = elems[i][0][0]
        label = elems[i][1][0]
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(image.astype("uint8"))
        idx = label
        if categorical:
            idx = np.argmax(idx)                
        plt.title(class_names[idx])
        plt.axis("off")
            
def class_distribution(dataset, class_names):
    class_values = []
    categorical = is_categorical(dataset)
    for element in dataset.as_numpy_iterator():
        for i in range(len(element[1])):
            y = element[1][i]
            if categorical:
                y = np.argmax(y)
            class_values.append(class_names[int(y)])

    class_n, frequency = np.unique(np.array(class_values), return_counts=True)
    df = pd.DataFrame(frequency, class_n, columns=["count"])
    return df

# frequency bar chart
def show_bar_chart(dataset, class_names, title):
    df = class_distribution(dataset, class_names)
    df.plot.bar()
    plt.title(title)
    plt.show()


def evaluate(model, test_dataset):
    pred_y = model.predict(test_dataset)
    pred_y = np.argmax(pred_y, axis=1)
    
    true_y = np.asarray(list(test_dataset.map(lambda x,y: y).unbatch().as_numpy_iterator()))
    if(is_categorical(test_dataset)):
        true_y = np.argmax(true_y, axis=1)
    
    #get confusion matrix
    cmatrix = confusion_matrix(true_y,pred_y)
    # get report
    report = classification_report(true_y,pred_y)
    
    return cmatrix, report 
    
def show_evaluation(history, conf_matrix, report):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Loss')
    plt.ylabel('value')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Accuracy')
    plt.ylabel('value')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    
    graph = ConfusionMatrixDisplay(conf_matrix)
    graph.plot()
    plt.show()
    print(report)

In [ ]:
# Set Image pre-processing parameters
AUTOTUNE = tf.data.experimental.AUTOTUNE  #AUTOTUNE= -1 ???
IMG_SIZE = (300, 300)
SHUFFLE=True
SEED=42
LABEL_MODE = "int"
COLOR_MODE = "rgb"
VAL_SPLIT = 0.2
TEST_SPLIT = 0.2

directory = "/kaggle/input/garbage-classification/Garbage classification/Garbage classification/"
dataset = image_dataset_from_directory(directory,
                                       batch_size=1,
                                       shuffle=SHUFFLE,
                                       image_size=IMG_SIZE,
                                       label_mode=LABEL_MODE,
                                       color_mode=COLOR_MODE,
                                       seed=SEED)

train_dataset, validation_dataset, test_dataset = get_dataset_partitions_tf(dataset)

CLASS_NAMES = dataset.class_names
NUM_CLASSES = len(CLASS_NAMES)

DEVELOPMENT = False
if DEVELOPMENT:
    seed = 12
    train_dataset.shuffle(train_dataset.cardinality(),seed=seed)
    train_dataset = train_dataset.take(480)
    validation_dataset.shuffle(validation_dataset.cardinality(),seed=seed)
    validation_dataset = validation_dataset.take(160)
    test_dataset.shuffle(test_dataset.cardinality(),seed=seed)
    test_dataset = test_dataset.take(160)

In [ ]:
print(f"{CLASS_NAMES}")

In [ ]:
print("sparse categorical")
print(f"categorical? {is_categorical(dataset)}")
for elem in dataset.as_numpy_iterator():
    print(elem[0].shape)
    show_image(elem, CLASS_NAMES)
    break

In [ ]:
visualize_dataset(dataset, CLASS_NAMES, "Normal dataset")

In [ ]:
visualize_dataset(train_dataset, CLASS_NAMES, "Training dataset")

In [ ]:
# frequency in train dataset
show_bar_chart(train_dataset, CLASS_NAMES, f"Training data: Frequency")
show_bar_chart(validation_dataset, CLASS_NAMES, f"Validation data: Frequency")
show_bar_chart(test_dataset, CLASS_NAMES, f"Test data: Frequency")

In [ ]:
def data_augmenter():
    '''
    Create a Sequential model composed of 2 layers
    Returns:
        tf.keras.Sequential
    '''

    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(RandomFlip("horizontal"))
    data_augmentation.add(RandomRotation(0.2))

    return data_augmentation

# Train models
We will train different models to set which ones fit our dataset better, basically using transfer learning models, with some improvements

## First model
Transfer learning: VGG16

In [ ]:
# Constants & parameters
VGG16_SIZE = 224
base_learning_rate = 0.01
optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate)
loss = tf.keras.losses.CategoricalCrossentropy()
initial_epochs = 15
BATCH_SIZE = 32
input_shape = (VGG16_SIZE, VGG16_SIZE, 3)
data_augmentation=data_augmenter()

### Preprocessing of dataset
model 01

In [ ]:
# pre_process data
def preprocess_data01(
    train_dataset, 
    validation_dataset, 
    test_dataset,
    batch_size=BATCH_SIZE
):

    # rebatch
    train_ds = train_dataset.rebatch(batch_size)
    val_ds = validation_dataset.rebatch(batch_size)
    test_ds = test_dataset.rebatch(batch_size)

    # categorise
    train_ds = train_ds.map(lambda x,y: (x, to_categorical(y, NUM_CLASSES)) )
    val_ds = val_ds.map(lambda x,y: (x, to_categorical(y, NUM_CLASSES)) )
    test_ds = test_ds.map(lambda x,y: (x, to_categorical(y, NUM_CLASSES)) )

    # resize
    resizing = Resizing(VGG16_SIZE, VGG16_SIZE)
    train_ds = train_ds.map(lambda x,y: (resizing(x), y))
    val_ds = val_ds.map(lambda x,y: (resizing(x), y))
    test_ds = test_ds.map(lambda x,y: (resizing(x), y))

    #rescale
    normalization = Rescaling(1.0/255)
    train_ds = train_ds.map(lambda x,y: (normalization(x), y))
    val_ds = val_ds.map(lambda x,y: (normalization(x), y))

    # prefetch
    train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
    test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)
    
    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = preprocess_data01(train_dataset, validation_dataset, test_dataset, batch_size=BATCH_SIZE)

### Model creation

In [ ]:
#Load the base pretrained VGG16 model. 
def first_model(input_shape, data_augmentation):
    base_model = vgg16.VGG16(
                    input_shape=(VGG16_SIZE, VGG16_SIZE, 3),
                    include_top=False,
                    weights='imagenet'
    )
    base_model.trainable = False

    inputs = Input(shape=input_shape)
    x = data_augmentation(inputs) 
    x = vgg16.preprocess_input(x) 
    x = base_model(x, training=False)
    x = Flatten()(x)  
    x = Dense(8, activation='relu', name = 'FC01_32_relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(len(CLASS_NAMES), activation="softmax")(x)

    model = Model(inputs, x, name="1st_model")
    return model

In [ ]:
model01 = first_model(input_shape, data_augmentation)

In [ ]:
model01.summary()

### Model compilation

In [ ]:
model01.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

### Model train

In [ ]:
history01 = model01.fit(
    train_ds,
    validation_data=val_ds,
    epochs=initial_epochs,
)

### Model evaluation

In [ ]:
cmatrix01, report01 = evaluate(model01, test_ds)

In [ ]:
show_evaluation(history01, cmatrix01, report01)

## Second Model
Transfer learning: MobileNetV2

In [ ]:
# Constants & parameters
base_learning_rate = 0.01
optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate)
loss = tf.keras.losses.CategoricalCrossentropy()
initial_epochs = 15
input_shape = IMG_SIZE + (3,)
BATCH_SIZE = 32
data_augmentation=data_augmenter()

In [ ]:
# pre_process data
# pre_process data
def preprocess_data02(
    train_dataset, 
    validation_dataset, 
    test_dataset,
    batch_size=BATCH_SIZE
):

    # rebatch
    train_ds = train_dataset.rebatch(batch_size)
    val_ds = validation_dataset.rebatch(batch_size)
    test_ds = test_dataset.rebatch(batch_size)

    # categorise
    train_ds = train_ds.map(lambda x,y: (x, to_categorical(y, num_classes=NUM_CLASSES)) )
    val_ds = val_ds.map(lambda x,y: (x, to_categorical(y, num_classes=NUM_CLASSES)) )
    test_ds = test_ds.map(lambda x,y: (x, to_categorical(y, num_classes=NUM_CLASSES)) )

    #rescale
    normalization = Rescaling(1.0/255)
    train_ds = train_ds.map(lambda x,y: (normalization(x), y))
    val_ds = val_ds.map(lambda x,y: (normalization(x), y))

    # prefetch
    train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
    test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)
    
    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = preprocess_data02(train_dataset, validation_dataset, test_dataset, batch_size=BATCH_SIZE)

In [ ]:
def second_model(input_shape, data_augmentation):
    base_model = mobilenet_v2.MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False
    
    inputs = Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = mobilenet_v2.preprocess_input(x) 
    x = base_model(x, training=False)
    x = GlobalMaxPooling2D()(x)
    x = Dropout(0.2)(x)
    prediction_layer = Dense(len(CLASS_NAMES), activation="softmax")
    outputs = prediction_layer(x)
    model = Model(inputs, outputs, name="2nd_model")
    return model

In [ ]:
model02 = second_model(input_shape, data_augmentation)

In [ ]:
model02.summary()

In [ ]:
model02.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])

In [ ]:
history02 = model02.fit(
    train_ds,
    validation_data=val_ds,
    epochs=initial_epochs,
)

In [ ]:
cmatrix02, report02 = evaluate(model02, test_ds)

In [ ]:
show_evaluation(history02, cmatrix02, report02)

## Model 03
Transfer learning: ResNet50

In [ ]:
# Constants & parameters
base_learning_rate = 0.01
optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate)
loss = tf.keras.losses.CategoricalCrossentropy()
initial_epochs = 15
input_shape = IMG_SIZE + (3,)
BATCH_SIZE = 32
data_augmentation=data_augmenter()

In [ ]:
# pre_process data
def preprocess_data03(
    train_dataset, 
    validation_dataset, 
    test_dataset,
    batch_size=BATCH_SIZE
):

    # rebatch
    train_ds = train_dataset.rebatch(batch_size)
    val_ds = validation_dataset.rebatch(batch_size)
    test_ds = test_dataset.rebatch(batch_size)

    # categorise
    train_ds = train_ds.map(lambda x,y: (x, to_categorical(y, num_classes=NUM_CLASSES)) )
    val_ds = val_ds.map(lambda x,y: (x, to_categorical(y, num_classes=NUM_CLASSES)) )
    test_ds = test_ds.map(lambda x,y: (x, to_categorical(y, num_classes=NUM_CLASSES)) )

    #rescale
    normalization = Rescaling(1.0/255)
    train_ds = train_ds.map(lambda x,y: (normalization(x), y))
    val_ds = val_ds.map(lambda x,y: (normalization(x), y))

    # prefetch
    train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
    test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)
    
    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = preprocess_data03(train_dataset, validation_dataset, test_dataset, batch_size=BATCH_SIZE)

In [ ]:
def third_model(input_shape, data_augmentation):
    base_model = resnet50.ResNet50(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False
    
    inputs = Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = resnet50.preprocess_input(x) 
    x = base_model(x, training=False)
    x = GlobalMaxPooling2D()(x)
    x = Dropout(0.2)(x)
    prediction_layer = Dense(len(CLASS_NAMES), activation="softmax")
    outputs = prediction_layer(x)
    model = Model(inputs, outputs, name="3rd_model")
    return model

In [ ]:
model03 = third_model(input_shape, data_augmentation)

In [ ]:
model03.summary()

In [ ]:
model03.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])

In [ ]:
history03 = model03.fit(
    train_ds,
    validation_data=val_ds,
    epochs=initial_epochs,
)

In [ ]:
cmatrix03, report03 = evaluate(model03, test_ds)

In [ ]:
show_evaluation(history03, cmatrix03, report03)

## Conclusions
Most models overfit the dataset, and it is not considered appropriated. Considering similar approaches from different sources, we will try to train with similar architecture from the previous ones, but with training parameters.

## Model 04
Convolutional architecture: MobileNet

In [ ]:
# Constants & parameters
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.CategoricalCrossentropy()
initial_epochs = 20
input_shape = IMG_SIZE + (3,)
BATCH_SIZE = 32
data_augmentation=data_augmenter()

In [ ]:
# pre_process data
def preprocess_data04(
    train_dataset, 
    validation_dataset, 
    test_dataset,
    batch_size=BATCH_SIZE
):

    # rebatch
    train_ds = train_dataset.rebatch(batch_size)
    val_ds = validation_dataset.rebatch(batch_size)
    test_ds = test_dataset.rebatch(batch_size)

    # categorise
    train_ds = train_ds.map(lambda x,y: (x, to_categorical(y, num_classes=NUM_CLASSES)) )
    val_ds = val_ds.map(lambda x,y: (x, to_categorical(y, num_classes=NUM_CLASSES)) )
    test_ds = test_ds.map(lambda x,y: (x, to_categorical(y, num_classes=NUM_CLASSES)) )

    #rescale
    normalization = Rescaling(1.0/255)
    train_ds = train_ds.map(lambda x,y: (normalization(x), y))
    val_ds = val_ds.map(lambda x,y: (normalization(x), y))

    # prefetch
    train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
    test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)
    
    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = preprocess_data04(train_dataset, validation_dataset, test_dataset, batch_size=BATCH_SIZE)

In [ ]:
def fourth_model(input_shape, data_augmentation):
    base_model = mobilenet_v2.MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet',
        classifier_activation='softmax',
    )
    base_model.trainable = True
    
    inputs = Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = mobilenet_v2.preprocess_input(x) 
    x = base_model(x, training=True)
    x = GlobalAveragePooling2D()(x)
    x = layers.Dense(128)(x) 
    x = layers.Activation(relu)(x) 
    x = layers.Dense(64)(x)
    x = layers.Activation(relu)(x)
    outputs = Dense(len(CLASS_NAMES), activation=softmax)(x)
    model = Model(inputs, outputs, name="4th_model")
    return model

In [ ]:
model04 = fourth_model(input_shape, data_augmentation)

In [ ]:
model04.summary()

In [ ]:
model04.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])

In [ ]:
cmatrix04, report04 = evaluate(model04, test_ds)

In [ ]:
show_evaluation(history04, cmatrix04, report04)

## Model 5
Model: ResNet50V2

In [ ]:
# Constants & parameters
base_learning_rate = 0.01
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.CategoricalCrossentropy()
initial_epochs = 20
input_shape = IMG_SIZE + (3,)
BATCH_SIZE = 32
data_augmentation=data_augmenter()

In [ ]:
# pre_process data
def preprocess_data05(
    train_dataset, 
    validation_dataset, 
    test_dataset,
    batch_size
):

    # rebatch
    train_ds = train_dataset.rebatch(batch_size)
    val_ds = validation_dataset.rebatch(batch_size)
    test_ds = test_dataset.rebatch(batch_size)

    # categorise
    train_ds = train_ds.map(lambda x,y: (x, to_categorical(y, num_classes=NUM_CLASSES)) )
    val_ds = val_ds.map(lambda x,y: (x, to_categorical(y, num_classes=NUM_CLASSES)) )
    test_ds = test_ds.map(lambda x,y: (x, to_categorical(y, num_classes=NUM_CLASSES)) )

    #rescale
    normalization = Rescaling(1.0/255)
    train_ds = train_ds.map(lambda x,y: (normalization(x), y))
    val_ds = val_ds.map(lambda x,y: (normalization(x), y))

    # prefetch
    train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
    test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)
    
    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = preprocess_data05(train_dataset, validation_dataset, test_dataset, batch_size=BATCH_SIZE)

In [ ]:
def fifth_model(input_shape, data_augmentation):
    base_model = resnet_v2.ResNet50V2(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = True
    
    inputs = Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = resnet_v2.preprocess_input(x) 
    x = base_model(x, training=True)
    x = GlobalAveragePooling2D()(x)
    x = layers.Dense(128)(x) 
    x = layers.Activation(relu)(x) 
    x = layers.Dense(64)(x)
    x = layers.Activation(relu)(x)
    outputs = Dense(len(CLASS_NAMES), activation="softmax")(x)
    model = Model(inputs, outputs, name="5th_model")
    return model

In [ ]:
model05 = fifth_model(input_shape, data_augmentation)

In [ ]:
model05.summary()

In [ ]:
model05.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])

In [ ]:
history05 = model05.fit(
    train_ds,
    validation_data=val_ds,
    epochs=initial_epochs,
)

In [ ]:
cmatrix05, report05 = evaluate(model05, test_ds)

In [ ]:
show_evaluation(history05, cmatrix05, report05)

## Model 06

In [ ]:
# Constants & parameters
optimizer = tf.keras.optimizers.Adam()
loss = "sparse_categorical_crossentropy"
initial_epochs = 20
input_shape = IMG_SIZE + (3,)
BATCH_SIZE = 32
data_augmentation=data_augmenter()

In [ ]:
# pre_process data
def preprocess_data06(
    train_dataset, 
    validation_dataset, 
    test_dataset,
    batch_size
):

    # rebatch
    train_ds = train_dataset.rebatch(batch_size)
    val_ds = validation_dataset.rebatch(batch_size)
    test_ds = test_dataset.rebatch(batch_size)
    
    # reshape
    

    #rescale
    normalization = Rescaling(1.0/255)
    train_ds = train_ds.map(lambda x,y: (normalization(x), y))
    val_ds = val_ds.map(lambda x,y: (normalization(x), y))

    # prefetch
    train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
    test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)
    
    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = preprocess_data06(train_dataset, validation_dataset, test_dataset, batch_size=BATCH_SIZE)

In [ ]:
def sixth_model(input_shape, data_augmentation):
    base_model = resnet_v2.ResNet50V2(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = True
    
    inputs = Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = resnet_v2.preprocess_input(x) 
    x = base_model(x, training=True)
    x = GlobalAveragePooling2D()(x)
    x = layers.Dense(128)(x) 
    x = layers.Activation(relu)(x) 
    x = layers.Dense(64)(x)
    x = layers.Activation(relu)(x)
    outputs = Dense(len(CLASS_NAMES), activation="softmax")(x)
    model = Model(inputs, outputs, name="6th_model")
    return model

In [ ]:
model06 = sixth_model(input_shape, data_augmentation)

In [ ]:
model06.summary()

In [ ]:
model06.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])

In [ ]:
history06 = model06.fit(
    train_ds,
    validation_data=val_ds,
    epochs=initial_epochs,
)

In [ ]:
cmatrix06, report06 = evaluate(model06, test_ds)

In [ ]:
show_evaluation(history06, cmatrix06, report06)

## Model 07

In [ ]:
# Constants & parameters
optimizer = tf.keras.optimizers.Adam()
loss = "sparse_categorical_crossentropy"
initial_epochs = 20
input_shape = IMG_SIZE + (3,)
BATCH_SIZE = 32
data_augmentation=data_augmenter()

In [ ]:
# pre_process data
def preprocess_data07(
    train_dataset, 
    validation_dataset, 
    test_dataset,
    batch_size=BATCH_SIZE
):

    # rebatch
    train_ds = train_dataset.rebatch(batch_size)
    val_ds = validation_dataset.rebatch(batch_size)
    test_ds = test_dataset.rebatch(batch_size)

    #rescale
    normalization = Rescaling(1.0/255)
    train_ds = train_ds.map(lambda x,y: (normalization(x), y))
    val_ds = val_ds.map(lambda x,y: (normalization(x), y))

    # prefetch
    train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
    test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)
    
    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = preprocess_data07(train_dataset, validation_dataset, test_dataset, batch_size=BATCH_SIZE)

In [ ]:
def seventh_model(input_shape, data_augmentation):
    base_model = mobilenet_v2.MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet',
        classifier_activation='softmax',
    )
    base_model.trainable = True
    
    inputs = Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = mobilenet_v2.preprocess_input(x) 
    x = base_model(x, training=True)
    x = GlobalAveragePooling2D()(x)
    x = layers.Dense(128)(x) 
    x = layers.Activation(relu)(x) 
    x = layers.Dense(64)(x)
    x = layers.Activation(relu)(x)
    outputs = Dense(len(CLASS_NAMES), activation=softmax)(x)
    model = Model(inputs, outputs, name="7th_model")
    return model

In [ ]:
model07 = seventh_model(input_shape, data_augmentation)

In [ ]:
model07.summary()

In [ ]:
model07.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])

In [ ]:
history07 = model07.fit(
    train_ds,
    validation_data=val_ds,
    epochs=initial_epochs,
)

In [ ]:
cmatrix07, report07 = evaluate(model07, test_ds)

In [ ]:
show_evaluation(history07, cmatrix07, report07)